In [1]:
import pymc as pm
import arviz as az
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import ipywidgets
import bambi as bmb

# 忽略不必要的警告
import warnings
warnings.filterwarnings("ignore")


# 研究假设  
假设 1a：在西方集群中，个人力量对道德判断有影响。  

假设 1b：如果个人力量的影响在文化上是普遍的，那么个人力量也会对南部和东部文化集群的道德可接受性评级产生影响。  

假设 2a：在西方集群中，个人力量和意图之间存在相互作用。更具体地说，与没有个人力量时相比，存在个人力量时的意图因素更大。  

假设 2b：如果这种影响在文化上是普遍的，那么在南部和东部文化集群中也存在影响。  


![Image Name](https://cdn.kesci.com/upload/s6j30bifr6.png?imageView2/0/w/960/h/960)  



![Image Name](https://cdn.kesci.com/upload/s6j32y9gj1.png?imageView2/0/w/960/h/960)  



# study1  
假设 1a：在西方集群中，个人力量对道德判断有影响。  

假设 1b：如果个人力量的影响在文化上是普遍的，那么个人力量也会对南部和东部文化集群的道德可接受性评级产生影响。

In [2]:
# study1数据处理
df_raw = pd.read_csv('/home/mw/input/dcsy9104/sj1.csv', encoding='ISO-8859-1')
df1a=df_raw[df_raw['include_study1a']==True]
df1a = df1a[["exp", "trolley_rate", "trolley_resp","Region"]]
df1a["index"] = range(len(df1a))
df1a = df1a.set_index("index")
df1a["personal_force"] =  np.where(df1a['exp'] == 1, 1, 0)
df1a

,exp,trolley_rate,trolley_resp,Region,personal_force
index,,,,,
0,2.0,1.0,2.0,Eastern,0
1,2.0,6.0,1.0,Eastern,0
2,1.0,2.0,2.0,Eastern,1
3,1.0,9.0,1.0,Eastern,1
4,1.0,6.0,1.0,Eastern,1
...,...,...,...,...,...
1564,2.0,2.0,2.0,Western,0
1565,1.0,2.0,2.0,Western,1
1566,2.0,1.0,2.0,Western,0


## study1数据可视化

In [3]:
# study1数据可视化
# 按照 region 分类绘制的子图列表
fig, axes = plt.subplots(nrows=1, ncols=len(df1a["Region"].unique()), figsize=(15,5))

# 遍历每个 region，分别绘制 countplot
for i, region in enumerate(df1a["Region"].unique()):
    sns.countplot(data=df1a[df1a["Region"]==region], x="trolley_rate", hue="personal_force", ax=axes[i])
    axes[i].set_xlabel("trolley_rate")
    axes[i].set_ylabel("Count")
    axes[i].set_title(region)
    sns.despine(ax=axes[i])

plt.tight_layout()

<Figure size 1500x500 with 3 Axes>

In [4]:
fig, axes = plt.subplots(nrows=1, ncols=len(df1a["Region"].unique()), figsize=(15,5))
for i, region in enumerate(df1a["Region"].unique()):
    sns.boxplot(data=df1a[df1a["Region"]==region], x="personal_force", y="trolley_rate", ax=axes[i])
    axes[i].set_xlabel("personal_force")
    axes[i].set_ylabel("trolley_rate")
    axes[i].set_title(region)
    sns.despine(ax=axes[i])

plt.tight_layout()


<Figure size 1500x500 with 3 Axes>

In [5]:
#选取西方文化背景下的被试
df1a_western=df1a[df1a['Region']=="Western"]
df1a_western["index"] = range(len(df1a_western))
df1a_western = df1a_western.set_index("index")
df1a_western

,exp,trolley_rate,trolley_resp,Region,personal_force
index,,,,,
0,1.0,4.0,2.0,Western,1
1,1.0,6.0,1.0,Western,1
2,1.0,2.0,2.0,Western,1
3,1.0,1.0,2.0,Western,1
4,1.0,6.0,1.0,Western,1
...,...,...,...,...,...
561,2.0,2.0,2.0,Western,0
562,1.0,2.0,2.0,Western,1
563,2.0,1.0,2.0,Western,0


### 模型1定义  
###   1. 自变量： 个人力量  
###   2. 因变量：道德可接受度评级  
###   3. 数据关系：  
$$  
 
\begin{array}{lcrl}  

\text{data:} & \hspace{.05in} & Y_i|\beta_0,\beta_1,\sigma & \stackrel{ind}{\sim} N(\mu_i, \; \sigma^2)  \;\; \text{ with } \;\; \mu_i = \beta_0 +\beta_1X \\  
\text{priors:} & & \beta_{0}  & \sim N\left(4, 1^2 \right)  \\  
                    & & \beta_1  & \sim N\left(0, 1^2 \right) \\  
                    & & \sigma   & \sim \text{Exp}(0.6)  .\\  
\end{array}  

$$  

In [6]:
with pm.Model() as model1:

    beta_0 = pm.Normal("beta_0", mu=4, sigma=1)          #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=-0.5, sigma=1)         #定义beta_1
    sigma = pm.Exponential("sigma", 0.6)                  #定义sigma

    model1.add_coord('obs_id',df1a_western.index, mutable=True)
    personal_force = pm.MutableData("personal_force",df1a_western.personal_force,dims="obs_id")                     #x是personal_force
    mu = pm.Deterministic("mu", beta_0 + beta_1*personal_force , dims="obs_id") #定义mu，自变量与先验结合

    #mu = beta_0 + beta_1 * personal_force                              #定义mu，将自变量与先验结合

    likelihood = pm.Normal("y_est", mu=mu, sigma=sigma, observed=df1a_western.trolley_rate,dims="obs_id")   #定义似然：预测值y符合N(mu, sigma)分布
                                                                                #通过 observed 传入实际数据

In [7]:
pm.model_to_graphviz(model1)

## model1 MCMC评估

In [8]:
with model1:
    model1_trace = pm.sample(draws=5000,            # 使用mcmc方法进行采样，draws为采样次数
                      tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                      chains=4,                     # 链数
                      discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                      random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, sigma]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 7 seconds.


In [9]:
model1_prior = pm.sample_prior_predictive(samples=50,
                                        model=model1,
                                        random_seed=84735)

Sampling: [beta_0, beta_1, sigma, y_est]


In [10]:
model1_prior

Inference data with groups:
	> prior
	> prior_predictive
	> observed_data
	> constant_data

In [11]:
az.plot_trace(model1_trace, var_names=["beta_0","beta_1",],
              figsize=(15,10),compact=False)
plt.show()

<Figure size 1500x1000 with 4 Axes>

In [12]:
az.plot_posterior(model1_trace, var_names=["beta_0","beta_1"])
plt.show()

<Figure size 1472x552 with 2 Axes>

In [13]:
az.summary(model1_trace, var_names=["beta_0","beta_1"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,3.758,0.123,3.531,3.992,0.001,0.001,10621.0,12074.0,1.0
beta_1,-0.594,0.171,-0.926,-0.283,0.002,0.001,10355.0,12656.0,1.0


## model1后验参数解释  
以下的结果显示：  

β0=3.758， 表明 X1 为 0即不需要对倒霉蛋用个人武力时，在西方文化背景下的被试认为这件事情从道德上可以接受的分数为3.758，(分数范围为1-9分)。  
β1=-0.594， 表明当在西方文化背景下的被试需要使用个人武力时，被试认为这件事情从道德上可以接受的分数将降低0.594，为3.164。  
 β1的94%HDI不包括0，说明个人武力能有效预测道德接受度变化的概率

## model1后验回归线预测

In [14]:
model1_trace

Inference data with groups:
	> posterior
	> sample_stats
	> observed_data
	> constant_data

In [15]:
for i in range(100):
    sns.lineplot(x = model1_trace.constant_data["personal_force"],
            y = model1_trace.posterior["mu"].stack(sample=("chain", "draw"))[:,i], 
            c="grey",
            alpha=0.4)
#设置x、y轴标题和总标题    
plt.xlabel("personal_force",
           fontsize=12)
plt.ylabel("probability of romantic",
           fontsize=12)
plt.suptitle("100 posterior plausible models",
           fontsize=14)
sns.despine()
plt.show()

<Figure size 640x480 with 1 Axes>

东方文化背景：  
![Image Name](https://cdn.kesci.com/upload/s6mk3johtm.png?imageView2/0/w/960/h/960)  
南方文化背景：  
![Image Name](https://cdn.kesci.com/upload/s6mk7vfds8.png?imageView2/0/w/960/h/960)  
由此验证了假设1b个人力量对于道德判断的影响具有文化普适性  


# study2  
假设 2a：在西方集群中，个人力量和意图之间存在相互作用。更具体地说，与没有个人力量时相比，存在个人力量时的意图因素更大。  

假设 2b：如果这种影响在文化上是普遍的，那么在南部和东部文化集群中也存在影响。  


## study2数据处理

In [16]:
df2a=df_raw[df_raw['include_study2a']==True]
df2a = df2a[["exp", "trolley_rate", "trolley_resp","Region"]]
df2a["index"] = range(len(df2a))
df2a = df2a.set_index("index")
#对数据进行重新编码
df2a["personal_force"] = pd.Series(np.where((df2a['exp'] == 4) | (df2a['exp'] == 6), 1, 0), index=df2a.index)
df2a["intension"] = pd.Series(np.where((df2a['exp'] == 4) | (df2a['exp'] == 5), 1, 0), index=df2a.index)
df2a

,exp,trolley_rate,trolley_resp,Region,personal_force,intension
index,,,,,,
0,5.0,5.0,1.0,Eastern,0,1
1,3.0,6.0,1.0,Eastern,0,0
2,4.0,8.0,1.0,Eastern,1,1
3,3.0,5.0,1.0,Eastern,0,0
4,6.0,1.0,2.0,Eastern,1,0
...,...,...,...,...,...,...
3979,4.0,1.0,2.0,Western,1,1
3980,6.0,6.0,1.0,Western,1,0
3981,6.0,1.0,2.0,Western,1,0


In [17]:
#选取西方文化背景下的被试
df2a_western=df2a[df2a['Region']=="Western"]
df2a_western["index"] = range(len(df2a_western))
df2a_western = df2a_western.set_index("index")
df2a_western

,exp,trolley_rate,trolley_resp,Region,personal_force,intension
index,,,,,,
0,6.0,4.0,1.0,Western,1,0
1,3.0,4.0,2.0,Western,0,0
2,5.0,4.0,2.0,Western,0,1
3,4.0,3.0,2.0,Western,1,1
4,4.0,2.0,2.0,Western,1,1
...,...,...,...,...,...,...
2966,4.0,1.0,2.0,Western,1,1
2967,6.0,6.0,1.0,Western,1,0
2968,6.0,1.0,2.0,Western,1,0


## study2数据可视化

In [18]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1) 
sns.barplot(data=df2a_western,
            x="personal_force",
            y="trolley_rate",
            hue="intension",
            alpha=0.6)
plt.xlabel("personal_force")
plt.ylabel("trolley_rate")
plt.xticks([0, 1], ['no', 'yes'])
sns.despine()

plt.subplot(1, 2, 2) 
sns.boxplot(data=df2a_western,
            x="personal_force",
            y="trolley_rate",
            hue="intension")
plt.xlabel("personal_force")
plt.ylabel("trolley_rate")
plt.xticks([0, 1], ['no', 'yes'])
plt.subplots_adjust(wspace=0.5)
plt.legend(title="Intension", loc="upper right")

plt.show()

<Figure size 1200x600 with 2 Axes>

## model2、model3定义

In [19]:
with pm.Model() as model2:
    model2.add_coord('obs_id',df2a_western.index, mutable=True)
    beta_0 = pm.Normal("beta_0", mu=4, sigma=1)        #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=-0.5, sigma=1)         #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=-0.5, sigma=1)         #定义beta_2
    sigma = pm.Exponential("sigma", 0.6)                 #定义sigma

    personal_force = pm.MutableData("personal_force",df2a_western.personal_force, dims="obs_id")      
    intension = pm.MutableData("intension",df2a_western.intension, dims="obs_id")        
    mu = pm.Deterministic("mu", beta_0 + 
                                beta_1*personal_force + 
                                beta_2*intension , dims="obs_id")      #定义mu，将自变量与先验结合

    likelihood = pm.Normal("y_est", mu=mu, sigma=sigma, observed=df2a_western.trolley_rate, dims="obs_id")

with pm.Model() as model3:
    model3.add_coord('obs_id',df2a_western.index, mutable=True)
    beta_0 = pm.Normal("beta_0", mu=4, sigma=1)        #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=-0.5, sigma=1)         #定义beta_1
    beta_2 = pm.Normal("beta_2", mu=-0.5, sigma=1)         #定义beta_2
    beta_3 = pm.Normal("beta_3", mu=0, sigma=1)          #定义beta_3
    sigma = pm.Exponential("sigma", 0.6)                 #定义sigma

    personal_force = pm.MutableData("personal_force",df2a_western.personal_force, dims="obs_id")      
    intension = pm.MutableData("intension",df2a_western.intension, dims="obs_id")        
    mu = pm.Deterministic("mu", beta_0 + 
                                beta_1*personal_force + 
                                beta_2*intension +
                                beta_3*personal_force*intension, dims="obs_id")      #定义mu，将自变量与先验结合

    likelihood = pm.Normal("y_est", mu=mu, sigma=sigma, observed=df2a_western.trolley_rate, dims="obs_id")

In [20]:
pm.model_to_graphviz(model2)

In [21]:
pm.model_to_graphviz(model3)

<table>  
        <tr>  
					<td>模型</td>  
           <td>model2</td>  
					 <td> model3</td>  
        </tr>  
        <tr>  
            <td>自变量</td>  
					<td>personal_force(二分变量)、intension(二分变量)</td>  
					<td>personal_force(二分变量)、intension(二分变量)</td>  
        </tr>  
	  <tr>  
            <td>自变量含义</td>  
					<td>`0` 表示no，`1`表示yes</td>  
					<td>`0` 表示no，`1`表示yes</td>  
   </tr>  
	 <tr>  
            <td>先验</td>  
					<td>  
β0 ~ N(4, 1) <br>  
β1 ~ N(-0.5, 1)  <br>  
β2 ~ N(-0.5, 1)  <br>  
σ ~ Exp(0.6) <br>  
		 </td>  
					<td>  
β0 ~ N(4, 1)  <br>  
β1 ~ N(-0.5, 1)  <br>  
β2 ~ N(-0.5, 1)  <br>  
β3 ~ N(0, 1)  <br>  
σ ~ Exp(0.6) <br>  
		 </td>  
  </tr>  
	  <tr>  
            <td>对比</td>  
					<td><img src="https://cdn.kesci.com/upload/rt/4CD774FB8E9D4DC8AAF61C709F27F8D3/s6gq6c9zbv.svg" alt="" width="100" height="200"> </td>  
			<td><img src="https://cdn.kesci.com/upload/rt/FF4152C637E044DB9FF886EF1839B08B/s6gq5o7tye.svg" alt="" width="100" height="200"> </td>  
   </tr>  
</table>  

## model2&model3 MCMC评估

In [22]:
#预计抽样半分钟
with model2:
    model2_trace = pm.sample(draws=5000,                   # 使用mcmc方法进行采样，draws为采样次数
                        tune=1000,                   # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                        chains=4,                    # 链数
                        discard_tuned_samples= True, # tune的结果将在采样结束后被丢弃
                        idata_kwargs={"log_likelihood": True},
                        random_seed=84735)
with model3:
    model3_trace = pm.sample(draws=5000,            # 使用mcmc方法进行采样，draws为采样次数
                      tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                      chains=4,                     # 链数
                      discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                      idata_kwargs={"log_likelihood": True},
                      random_seed=84735)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2, sigma]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 11 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_2, beta_3, sigma]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 17 seconds.


In [64]:
model2_trace

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data
	> constant_data

In [23]:
az.plot_trace(model2_trace, var_names=["beta_0","beta_1","beta_2"],
              figsize=(20,15),compact=False)
plt.show()

<Figure size 2000x1500 with 6 Axes>

In [24]:
az.plot_trace(model3_trace, var_names=["beta_0","beta_1","beta_2","beta_3"],
              figsize=(20,15),
              compact=False)
plt.show()

<Figure size 2000x1500 with 8 Axes>

## model2后验回归线预测

In [70]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import ScalarMappable

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xticks(range(1, 11))
ax.set_yticks(range(1, 11))
ax.set_zticks(range(1, 11))
# 获取数值数组
values = model2_trace.posterior["mu"].stack(sample=("chain", "draw"))

for i in range(400):
    ax.scatter(xs=model2_trace.constant_data["personal_force"],
               ys=model2_trace.constant_data["intension"],
               zs=values[:, i], 
               c=values[:, i],
               cmap="jet",
               alpha=0.4)

# 设置x、y轴标题和总标题    
ax.set_xlabel('personal_force')
ax.set_ylabel('intension')
ax.set_zlabel('trolley_rate')

# 创建颜色条带
cax = fig.add_axes([0.95, 0.1, 0.03, 0.8])  # 调整颜色条带位置和大小
sm = ScalarMappable(cmap="jet")
sm.set_array(values)
fig.colorbar(sm, cax=cax)
cax.set_ylabel('pi value')

plt.show()

<Figure size 1000x1000 with 2 Axes>

## model3后验回归线预测

In [71]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xticks(range(1, 11))
ax.set_yticks(range(1, 11))
ax.set_zticks(range(1, 11))
# 获取数值数组
values = model3_trace.posterior["mu"].stack(sample=("chain", "draw"))

for i in range(400):
    ax.scatter(xs=model3_trace.constant_data["personal_force"],
               ys=model3_trace.constant_data["intension"],
               zs=values[:, i], 
               c=values[:, i],
               cmap="jet",
               alpha=0.4)

# 设置x、y轴标题和总标题    
ax.set_xlabel('personal_force')
ax.set_ylabel('intension')
ax.set_zlabel('trolley_rate')

# 创建颜色条带
cax = fig.add_axes([0.95, 0.1, 0.03, 0.8])  # 调整颜色条带位置和大小
sm = ScalarMappable(cmap="jet")
sm.set_array(values)
fig.colorbar(sm, cax=cax)
cax.set_ylabel('pi value')

plt.show()

<Figure size 1000x1000 with 2 Axes>

In [25]:
az.summary(model2_trace, var_names=["beta_0","beta_1","beta_2"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,5.710,0.080,5.563,5.860,0.001,0.001,11521.0,12067.0,1.0
beta_1,-1.357,0.079,-1.510,-1.215,0.001,0.000,13436.0,12728.0,1.0
beta_2,-0.970,0.078,-1.113,-0.820,0.001,0.000,13625.0,13487.0,1.0


In [26]:
az.summary(model3_trace, var_names=["beta_0","beta_1","beta_2","beta_3"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,5.223,0.101,5.042,5.421,0.001,0.001,6110.0,9181.0,1.0
beta_1,-0.632,0.122,-0.866,-0.406,0.002,0.001,6031.0,9357.0,1.0
beta_2,-0.241,0.122,-0.478,-0.018,0.002,0.001,6175.0,8502.0,1.0
beta_3,-1.204,0.156,-1.503,-0.920,0.002,0.001,6371.0,9797.0,1.0


## 模型评估与比较

In [27]:
#预计比较半分钟
az.loo(model2_trace)
az.loo(model3_trace)
comparison_list = {
    "model2":model2_trace,
    "model3":model3_trace,
}
az.compare(comparison_list)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model3,0,-6396.515802,4.681642,0.000000,0.996269,33.458316,0.000000,False,log
model2,1,-6424.319174,3.698222,27.803372,0.003731,32.675772,7.441027,False,log


验证假设 2a：在西方集群中发现了个人力量和意图之间的相互作用

东方文化背景：  
![Image Name](https://cdn.kesci.com/upload/s6mkqd4gpt.png?imageView2/0/w/960/h/960)  
模型比较：  
![Image Name](https://cdn.kesci.com/upload/s6mkqp2eg8.png?imageView2/0/w/960/h/960)  

南方文化背景：  
![Image Name](https://cdn.kesci.com/upload/s6mkgy77f7.png?imageView2/0/w/960/h/960)  
模型比较：  
![Image Name](https://cdn.kesci.com/upload/s6mkhdqunf.png?imageView2/0/w/960/h/960)  

然而，东方文化背景下，个人力量与意图的交互作用并不显著，所以无法验证假设 2b  
不过这也与原文献的数据分析结果相符！  
(2) the interaction between intention and personal force was replicated in the** Southern and Western clusters**, finding people are less likely to support sacrificing one person’s life for the sake of saving the lives of several others, if they have both to intentionally engage in an action to do this and to use personal force  



# study3  


原文献通过对不同区域进行步骤相同的分析来说明结论具有文化普适性。我们更想进一步知道，除了文化普适性，个人力量的使用对道德可接受程度的影响是否在不同的国家间存在差异？  
可能1：道德可接受程度在不同国家间存在差异，但是个人力量对道德可接受程度的影响在不同国家间不存在差异  
可能2：国家只调节个人力量对道德可接受程度的影响，而各国家间道德可接受程度相当  
可能3：国家可能既影响道德可接受程度，又影响个人力量对道德可接受程度的影响。  


**模型0：** (complete_pooled_model)，普通线性模型，仅考虑个人力量对道德可接受程度的影响。  
**模型1：** (run_var_inter_model)，变化截距模型，在模型0的基础上考虑道德可接受程度在不同国家的变化。  
**模型2：**(run_var_slope_model)，变化斜率模型，在模型0的基础上不同国家间的个人力量影响的变化。  
**模型3：** (run_var_both_model)，变化截距和斜率模型，结合模型1和模型2，同时考虑国家对道德可接受程度以及个人武力的变化的影响。

In [28]:
df=df_raw[df_raw['include_study1a']==True]
df = df[["exp", "trolley_rate", "trolley_resp","Region","country3"]]
df["index"] = range(len(df))
df = df.set_index("index")
df["personal_force"] =  np.where(df['exp'] == 1, 1, 0)
df

,exp,trolley_rate,trolley_resp,Region,country3,personal_force
index,,,,,,
0,2.0,1.0,2.0,Eastern,ARE,0
1,2.0,6.0,1.0,Eastern,MYS,0
2,1.0,2.0,2.0,Eastern,MYS,1
3,1.0,9.0,1.0,Eastern,MYS,1
4,1.0,6.0,1.0,Eastern,CHN,1
...,...,...,...,...,...,...
1564,2.0,2.0,2.0,Western,GRC,0
1565,1.0,2.0,2.0,Western,GRC,1
1566,2.0,1.0,2.0,Western,GRC,0


In [29]:
sns.boxplot(data=df_raw,
            x="country3",
            y="trolley_rate")

plt.xticks(rotation=90) 
sns.despine()
plt.show()

<Figure size 640x480 with 1 Axes>

In [30]:
#为站点生成索引，为被试生成索引
df["country_idx"] = pd.factorize(df.country3)[0]
df["obs_id"] = range(len(df))

#设置索引，方便之后调用数据
df.set_index(['country3','obs_id'],inplace=True,drop=False)
df

exp  trolley_rate  trolley_resp   Region country3  \
country3 obs_id                                                      
ARE      0       2.0           1.0           2.0  Eastern      ARE   
MYS      1       2.0           6.0           1.0  Eastern      MYS   
         2       1.0           2.0           2.0  Eastern      MYS   
         3       1.0           9.0           1.0  Eastern      MYS   
CHN      4       1.0           6.0           1.0  Eastern      CHN   
...              ...           ...           ...      ...      ...   
GRC      1564    2.0           2.0           2.0  Western      GRC   
         1565    1.0           2.0           2.0  Western      GRC   
         1566    2.0           1.0           2.0  Western      GRC   
         1567    2.0           6.0           1.0  Western      GRC   
POL      1568    2.0           1.0           2.0  Western      POL   

                 personal_force  country_idx  obs_id  
country3 obs_id                                       
ARE      0                    0            0       0  
MYS      1                    0            1       1  
         2                    1            1       2  
         3                    1            1       3  
CHN      4                    1            2       4  
...                         ...          ...     ...  
GRC      1564                 0           32    1564  
         1565                 1           32    1565  
         1566                 0           32    1566  
         1567                 0           32    1567  
POL      1568                 0           25    1568  

[1569 rows x 8 columns]

## complete_pooled

In [31]:
coords = {"obs_id": df.obs_id}
with pm.Model(coords=coords) as complete_pooled_model:

    beta_0 = pm.Normal("beta_0", mu=4, sigma=1)                #定义beta_0          
    beta_1 = pm.Normal("beta_1", mu=0, sigma=1)                 #定义beta_1
    sigma = pm.Exponential("sigma", 1)                          #定义sigma

    x = pm.MutableData("x", df.personal_force, dims="obs_id")    #x是自变量压力水平

    mu = pm.Deterministic("mu",beta_0 + beta_1 * x, 
                          dims="obs_id")                        #定义mu，讲自变量与先验结合

    likelihood = pm.Normal("y_est", mu=mu, sigma=sigma, observed=df.trolley_rate,
                           dims="obs_id")                       #定义似然：预测值y符合N(mu, sigma)分布
                                                                #通过 observed 传入实际数据y 自我控制水平
    complete_trace = pm.sample(random_seed=84735, target_accept=0.99,idata_kwargs={"log_likelihood": True})

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 6 seconds.


In [32]:
pm.model_to_graphviz(complete_pooled_model)

In [33]:
az.summary(complete_trace,
           var_names=["~mu"],
           filter_vars="like")

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,3.994,0.078,3.843,4.139,0.002,0.001,1768.0,1877.0,1.0
beta_1,-0.849,0.107,-1.039,-0.634,0.003,0.002,1718.0,1956.0,1.0
sigma,2.149,0.038,2.079,2.221,0.001,0.001,2208.0,2032.0,1.0


## Hierarchical model with varying intercepts

In [34]:
# 定义函数来构建和采样模型
def run_var_inter_model(non_centered = False):

    #定义数据坐标，包括站点和观测索引
    coords = {"country3": df["country3"].unique(),
            "obs_id": df.obs_id}

    with pm.Model(coords=coords) as var_inter_model:
        #定义全局参数
        beta_0 = pm.Normal("beta_0", mu=4, sigma=1)
        beta_0_sigma = pm.Exponential("beta_0_sigma", 1)
        beta_1 = pm.Normal("beta_1", mu=-1, sigma=1)
        sigma_y = pm.Exponential("sigma_y", 1) 

        #传入自变量、获得观测值对应的站点映射
        x = pm.MutableData("x", df.personal_force, dims="obs_id")
        country3 = pm.MutableData("country3", df.country_idx, dims="obs_id") 
        
        #选择不同的模型定义方式
        if non_centered:
            beta_0_offset = pm.Normal("beta_0_offset", 0, sigma=1, dims="country3")
            beta_0j = pm.Deterministic("beta_0j", beta_0 + beta_0_offset * beta_0_sigma, dims="country3")
        else:
            beta_0j = pm.Normal("beta_0j", mu=beta_0, sigma=beta_0_sigma, dims="country3")

        #线性关系
        mu = pm.Deterministic("mu", beta_0j[country3]+beta_1*x, dims="obs_id")

        # 定义 likelihood
        likelihood = pm.Normal("y_est", mu=mu, sigma=sigma_y, observed=df.trolley_rate, dims="obs_id")

        var_inter_trace = pm.sample(draws=5000,           # 使用mcmc方法进行采样，draws为采样次数
                            tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                            chains=4,                     # 链数
                            discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                            random_seed=84735,
                            target_accept=0.99,
                            idata_kwargs={"log_likelihood": True}
                            )
    
    return var_inter_model, var_inter_trace

In [35]:
var_inter_model, var_inter_trace = run_var_inter_model(non_centered = True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_0_sigma, beta_1, sigma_y, beta_0_offset]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 50 seconds.


In [36]:
pm.model_to_graphviz(var_inter_model)

In [37]:
var_inter_para = az.summary(var_inter_trace,
           var_names=["~mu","~_sigma","~_offset","~sigma_"],
           filter_vars="like")
var_inter_para

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,4.113,0.149,3.847,4.406,0.002,0.002,4658.0,8179.0,1.0
beta_1,-0.898,0.105,-1.094,-0.700,0.001,0.001,21314.0,15320.0,1.0
beta_0j[ARE],2.605,0.326,1.983,3.208,0.002,0.002,18915.0,14167.0,1.0
beta_0j[MYS],4.449,0.426,3.633,5.234,0.003,0.002,22085.0,14503.0,1.0
beta_0j[CHN],4.028,0.230,3.593,4.453,0.001,0.001,37100.0,15997.0,1.0
beta_0j[IND],4.269,0.315,3.671,4.856,0.002,0.001,33833.0,15689.0,1.0
beta_0j[LBN],3.853,0.600,2.714,4.990,0.004,0.003,23154.0,14364.0,1.0
beta_0j[MKD],4.033,0.244,3.572,4.483,0.001,0.001,36176.0,16332.0,1.0
beta_0j[THA],4.181,0.435,3.374,5.009,0.002,0.002,33703.0,14596.0,1.0
beta_0j[PAK],4.384,0.232,3.945,4.813,0.001,0.001,29312.0,15148.0,1.0


In [61]:
az.plot_forest(var_inter_trace,
           var_names=["~mu", "~sigma", "~offset", "~beta_1"],
           filter_vars="like",
           combined = True
           )

plt.show()

<Figure size 600x840 with 1 Axes>

## Hierarchical model with varying slopes

In [38]:
# 定义函数来构建和采样模型
def run_var_slope_model(non_centered = False):

    #定义数据坐标，包括站点和观测索引
    coords = {"country3": df["country3"].unique(),
            "obs_id": df.obs_id}

    with pm.Model(coords=coords) as var_slope_model:
        #定义全局参数
        beta_0 = pm.Normal("beta_0", mu=4, sigma=1)
        beta_1 = pm.Normal("beta_1", mu=-1, sigma=1) 
        beta_1_sigma = pm.Exponential("beta_1_sigma", 1)
        sigma_y = pm.Exponential("sigma_y", 1) 

        #传入自变量、获得观测值对应的站点映射
        x = pm.MutableData("x", df.personal_force, dims="obs_id")
        country3 = pm.MutableData("country3", df.country_idx, dims="obs_id") 

        #选择不同的模型定义方式
        if non_centered:
            beta_1_offset = pm.Normal("beta_1_offset", 0, sigma=1, dims="country3")
            beta_1j = pm.Deterministic("beta_1j", beta_1 + beta_1_offset * beta_1_sigma, dims="country3")
        else:
            beta_1j = pm.Normal("beta_1j", mu=beta_1, sigma=beta_1_sigma, dims="country3")

        #线性关系
        mu = pm.Deterministic("mu", beta_0+beta_1j[country3]*x, dims="obs_id")

        # 定义 likelihood
        likelihood = pm.Normal("y_est", mu=mu, sigma=sigma_y, observed=df.trolley_rate, dims="obs_id")

        var_slope_trace = pm.sample(draws=5000,           # 使用mcmc方法进行采样，draws为采样次数
                            tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                            chains=4,                     # 链数
                            discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                            random_seed=84735,
                            target_accept=0.99,
                            idata_kwargs={"log_likelihood": True})
    
    return var_slope_model, var_slope_trace

In [39]:
var_slope_model, var_slope_trace = run_var_slope_model(non_centered = True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_1, beta_1_sigma, sigma_y, beta_1_offset]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 50 seconds.


In [40]:
pm.model_to_graphviz(var_slope_model)

In [49]:
var_slope_para = az.summary(var_slope_trace,
                            var_names=["beta_0","beta_1j"],
                            filter_vars="like")
var_slope_para

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,4.003,0.075,3.863,4.146,0.000,0.000,26559.0,15254.0,1.0
beta_1j[ARE],-1.638,0.406,-2.377,-0.857,0.003,0.002,16366.0,13501.0,1.0
beta_1j[MYS],-0.711,0.440,-1.528,0.145,0.003,0.002,28294.0,15704.0,1.0
beta_1j[CHN],-0.850,0.315,-1.461,-0.274,0.002,0.001,35959.0,15640.0,1.0
beta_1j[IND],-1.017,0.365,-1.714,-0.338,0.002,0.002,35331.0,15429.0,1.0
beta_1j[LBN],-0.940,0.532,-1.992,0.046,0.003,0.003,29791.0,14250.0,1.0
beta_1j[MKD],-0.854,0.291,-1.404,-0.309,0.002,0.001,32534.0,16263.0,1.0
beta_1j[THA],-0.667,0.447,-1.468,0.221,0.003,0.002,31839.0,15249.0,1.0
beta_1j[PAK],-0.371,0.298,-0.923,0.204,0.002,0.002,25756.0,15285.0,1.0
beta_1j[IRN],-0.490,0.562,-1.453,0.664,0.004,0.003,20244.0,13172.0,1.0


In [59]:
az.plot_forest(var_slope_trace,
           var_names=["~mu", "~sigma", "~offset", "~beta_0"],
           filter_vars="like",
           combined = True
           )
plt.show()

<Figure size 600x840 with 1 Axes>

## Hierarchical model with varying intercepts & slopes

In [42]:
# 定义函数来构建和采样模型
def run_var_both_model(non_centered = False):

    #定义数据坐标，包括站点和观测索引
    coords = {"country3": df["country3"].unique(),
            "obs_id": df.obs_id}

    with pm.Model(coords=coords) as model:
        #定义全局参数
        beta_0 = pm.Normal("beta_0", mu=0, sigma=50)
        beta_0_sigma = pm.Exponential("beta_0_sigma", 1)
        beta_1 = pm.Normal("beta_1", mu=0, sigma=5) 
        beta_1_sigma = pm.Exponential("beta_1_sigma", 1)
        sigma_y = pm.Exponential("sigma_y", 1) 

        #传入自变量、获得观测值对应的站点映射
        x = pm.MutableData("x", df.personal_force, dims="obs_id")
        country3 = pm.MutableData("site", df.country_idx, dims="obs_id") 

        #选择不同的模型定义方式
        if non_centered:
            beta_0_offset = pm.Normal("beta_0_offset", 0, sigma=1, dims="country3")
            beta_0j = pm.Deterministic("beta_0j", beta_0 + beta_0_offset * beta_0_sigma, dims="country3")
            beta_1_offset = pm.Normal("beta_1_offset", 0, sigma=1, dims="country3")
            beta_1j = pm.Deterministic("beta_1j", beta_1 + beta_1_offset * beta_1_sigma, dims="country3")
        else:
            beta_0j = pm.Normal("beta_0j", mu=beta_0, sigma=beta_0_sigma, dims="country3")
            beta_1j = pm.Normal("beta_1j", mu=beta_1, sigma=beta_1_sigma, dims="country3")

        #线性关系
        mu = pm.Deterministic("mu", beta_0j[country3]+beta_1j[country3]*x, dims="obs_id")

        # 定义 likelihood
        likelihood = pm.Normal("y_est", mu=mu, sigma=sigma_y, observed=df.trolley_rate, dims="obs_id")

        trace = pm.sample(draws=5000,           # 使用mcmc方法进行采样，draws为采样次数
                            tune=1000,                    # tune为调整采样策略的次数，可以决定这些结果是否要被保留
                            chains=4,                     # 链数
                            discard_tuned_samples= True,  # tune的结果将在采样结束后被丢弃
                            random_seed=84735,
                            target_accept=0.99,idata_kwargs={"log_likelihood": True}
                            )
    
    return model, trace

In [43]:
var_both_model, var_both_trace = run_var_both_model(non_centered = True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta_0, beta_0_sigma, beta_1, beta_1_sigma, sigma_y, beta_0_offset, beta_1_offset]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 104 seconds.


In [44]:
pm.model_to_graphviz(var_both_model)

In [45]:
pd.set_option("display.max_rows", 66)

In [46]:
var_both_para = az.summary(var_both_trace,
                            var_names=["beta_0j","beta_1j"],
                            filter_vars="like")
var_both_para

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0j[ARE],2.604,0.343,1.973,3.251,0.003,0.002,16826.0,15304.0,1.0
beta_0j[MYS],4.446,0.426,3.617,5.221,0.003,0.002,27678.0,15504.0,1.0
beta_0j[CHN],4.022,0.238,3.587,4.480,0.001,0.001,32819.0,15485.0,1.0
beta_0j[IND],4.297,0.324,3.672,4.894,0.002,0.001,30996.0,16007.0,1.0
beta_0j[LBN],3.855,0.595,2.726,4.980,0.004,0.003,28192.0,14649.0,1.0
beta_0j[MKD],4.028,0.258,3.553,4.517,0.002,0.001,29104.0,15828.0,1.0
beta_0j[THA],4.173,0.434,3.361,4.987,0.003,0.002,29489.0,15142.0,1.0
beta_0j[PAK],4.337,0.251,3.862,4.805,0.002,0.001,24897.0,15103.0,1.0
beta_0j[IRN],4.934,0.685,3.659,6.216,0.006,0.004,14529.0,12608.0,1.0
beta_0j[JPN],4.018,0.261,3.525,4.503,0.001,0.001,30783.0,15931.0,1.0


In [47]:
# 设置绘图坐标
figs, (ax1, ax2) = plt.subplots(1,2, figsize = (60,15))
# 绘制变化的截距
az.plot_forest(var_both_trace,
           var_names=["~mu", "~sigma", "~offset", "~beta_1"],
           filter_vars="like",
           combined = True,
           ax=ax1)
# 绘制变化的斜率
az.plot_forest(var_both_trace,
           var_names=["~mu", "~sigma", "~offset", "~beta_0"],
           filter_vars="like",
           combined = True,
           ax=ax2)
plt.show()

<Figure size 6000x1500 with 2 Axes>

## 评估后验预测

In [50]:
# 进行后验预测
complete_ppc = pm.sample_posterior_predictive(complete_trace, 
                                            model = complete_pooled_model,
                                            random_seed=84735)
var_inter_ppc = pm.sample_posterior_predictive(var_inter_trace,
                                                model = var_inter_model,
                                                random_seed=84735)
var_slope_ppc = pm.sample_posterior_predictive(var_slope_trace,
                                                model = var_slope_model,
                                                random_seed=84735)                                                                                       
var_both_ppc = pm.sample_posterior_predictive(var_both_trace, 
                                            model = var_both_model,
                                            random_seed=84735)

Sampling: [y_est]


Sampling: [y_est]


Sampling: [y_est]


Sampling: [y_est]


In [51]:
# 定义计算 MAE 函数
from statistics import median
def MAE(model_ppc):
    # 计算每个X取值下对应的后验预测模型的均值
    pre_x = model_ppc.posterior_predictive["y_est"].stack(sample=("chain", "draw"))
    pre_y_mean = pre_x.mean(axis=1).values

    # 提取观测值Y，提取对应Y值下的后验预测模型的均值
    MAE = pd.DataFrame({
        "rate_ppc_mean": pre_y_mean,
        "rate_original": model_ppc.observed_data.y_est.values
    })

    # 计算预测误差
    MAE["pre_error"] = abs(MAE["rate_original"] -\
                            MAE["rate_ppc_mean"])

    # 最后，计算预测误差的中位数
    MAE = median(MAE.pre_error)
    return MAE

In [52]:
# 定义
def counter_outlier(model_ppc, hdi_prob=0.95):
    # 将az.summary生成的结果存到hdi_multi这个变量中，该变量为数据框
    hdi = az.summary(model_ppc, kind="stats", hdi_prob=hdi_prob)
    lower = hdi.iloc[:,2].values
    upper = hdi.iloc[:,3].values

    # 将原数据中的分数合并，便于后续进行判断
    y_obs = model_ppc.observed_data["y_est"].values

    # 判断原数据中的分数是否在后验预测的95%可信区间内，并计数
    hdi["verify"] = (y_obs <= lower) | (y_obs >= upper)
    hdi["y_obs"] = y_obs
    hdi_num = sum(hdi["verify"])

    return hdi_num

In [53]:
# 将每个模型的PPC储存为列表
ppc_samples_list = [complete_ppc,  var_inter_ppc, var_slope_ppc, var_both_ppc]
model_names = ["完全池化",  "变化截距", "变化斜率", "变化截距、斜率"]

# 建立一个空列表来存储结果
results_list = []

# 遍历模型并计算MAE和超出95%hdi的值
for model_name, ppc_samples in zip(model_names, ppc_samples_list):
    outliers = counter_outlier(ppc_samples)
    MAEs = MAE(ppc_samples)
    results_list.append({'Model': model_name, 'MAE':MAEs, 'Outliers': outliers})

# 从结果列表创建一个DataFrame
results_df = pd.DataFrame(results_list)

results_df

,Model,MAE,Outliers
0,完全池化,1.931636,55
1,变化截距,1.584359,56
2,变化斜率,1.754810,55
3,变化截距、斜率,1.584685,57


## study3模型比较

In [48]:
az.loo(complete_trace)
az.loo(var_inter_trace)
az.loo(var_slope_trace)
az.loo(var_both_trace)
comparison_list = {
    "complete":complete_trace,
    "var_inter":var_inter_trace,
    "var_slope":var_slope_trace,
    "var_both":var_both_trace,
}
az.compare(comparison_list)

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
var_inter,0,-3396.060044,24.519920,0.000000,1.0,25.307931,0.000000,False,log
var_both,1,-3396.772399,27.208365,0.712355,0.0,25.318689,0.463808,False,log
var_slope,2,-3418.831372,16.444686,22.771328,0.0,24.893301,6.398886,False,log
complete,3,-3429.151480,2.751372,33.091436,0.0,24.537677,7.627680,False,log


### 模型比较的结果表明：道德可接受程度在不同国家间存在差异，但是个人武力对道德可接受程度的影响在不同国家间不存在差异。但是由于变化截距和变化斜率、截距模型的elpd非常非常接近，模型比较的结果只能作为参考。

### 总结：  
**研究1：** 原文献使用了t检验对研究1进行了分析，我们使用了Normal-Normal贝叶斯模型。得出结论：个人力量的使用对道德判断中道德可接受度的有显著影响，并且具有文化普适性（1a、1b）。【与文献结论一致】  

**研究2：** 对于研究二交互作用的分析，我们定义了两个模型，模型2不考虑交互作用，模型3考虑交互作用。模型比较的结果告诉我们：在西方、南方文化集群中，个人力量和意图对道德判断存在交互作用，但无法在东方文化集群中得到同样的结果（2a）【与文献结论一致】  

